In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf

In [3]:
try:
    conn = psycopg2.connect(host = conf.database_user.host, dbname=conf.database_user.dbname, user=conf.database_user.cslab, password=conf.database_user.password)
    cur = conn.cursor()
    cur.execute("\
select \
  a.id as q_id \
, a.posttypeid as q_posttypeid \
, a.acceptedanswerid as q_acceptedanswerid \
, a.parentid as q_parentid \
, a.creationdate as q_creationdate \
, a.score as q_score \
, a.viewcount as q_viewcount \
, a.owneruserid as q_owneruserid \
, a.title as q_title \
, a.tags as q_tags \
, a.answercount as q_answercount \
, a.commentcount as q_commentcount \
, b.id as  a_id \
, b.posttypeid as a_posttypeid \
, b.acceptedanswerid as  a_acceptedanswerid \
, b.parentid as  a_parentid \
, b.creationdate as  a_creationdate \
, b.score as  a_score \
, b.viewcount as  a_viewcount \
, b.owneruserid as a_owneruserid \
, b.title as  a_title \
, b.tags as  a_tags \
, b.answercount as a_commentcount \
, b.commentcount as a_commentcount \
, c.reputation as q_reputation \
, d.reputation as a_reputation \
  from posts a  \
     , posts b \
     , users c \
     , users d \
 where a.creationdate >= '2023-11-03' \
   and a.posttypeid ='1' \
   and b.parentid = a.id \
   and b.posttypeid='2' \
   and a.owneruserid is not null  \
   and b.owneruserid is not null  \
   and c.id = a.owneruserid  \
   and d.id = b.owneruserid;  \
                "
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)

AttributeError: module 'config.config' has no attribute 'host'

In [1]:
df = pd.DataFrame(rows, columns = [
'q_id','q_posttypeid','q_acceptedanswerid', 'q_parentid', 'q_creationdate', 'q_score', 'q_viewcount', 'q_owneruserid', 'q_title', 'q_tags', 'q_answercount', 'q_commentcount',
'a_id','a_posttypeid','a_acceptedanswerid', 'a_parentid', 'a_creationdate', 'a_score', 'a_viewcount', 'a_owneruserid', 'a_title', 'a_tags', 'a_answercount', 'a_commentcount',
'q_reputation','a_reputation'
])

NameError: name 'pd' is not defined

In [24]:
print(df.shape)

(44665, 26)


In [26]:
print(df['q_creationdate'].min())
print(df['a_creationdate'].max())

2023-11-03 00:03:41.097000
2023-12-03 10:15:43.250000


In [6]:
df_gephi = df[['q_owneruserid', 'a_owneruserid']]

In [7]:
df_gephi.columns = ['Source', 'Target']

In [8]:
node = np.unique(np.append(df_gephi['Source'].values, df_gephi['Target'].values))

In [9]:
# 질문자->답변자 로 edgelist 생성
edgelist = list(zip(df_gephi['Source'], df_gephi['Target']))

In [10]:
df_gephi.to_csv('../data/gephi.csv', index=False)

In [11]:
print(df['q_owneruserid'].min())
print(df['q_owneruserid'].max())

93
23031477


In [12]:
df_question = df_gephi['Source'].value_counts().reset_index()
df_answer = df_gephi['Target'].value_counts().reset_index()

In [13]:
df_gephi

,Source,Target
0,21804844,3144770
1,23031019,5479575
2,21118768,7318120
3,18000742,4959550
4,23029876,22821210
...,...,...
44660,5998081,3115457
44661,12886050,20037042
44662,22777750,14457599
44663,7904441,6780593


In [14]:
df_answer = df_answer.rename(columns = {'Target': 'user_id', 'count':'answer_cnt'})
df_question = df_question.rename(columns = {'Source': 'user_id', 'count':'question_cnt'})

In [15]:
df_qna = pd.merge(df_question, df_answer, on = 'user_id', how='outer')

In [16]:
df_qna.fillna(0, inplace=True)

In [17]:
df_qna

,user_id,question_cnt,answer_cnt
0,509627,28.0,4.0
1,139698,25.0,1.0
2,832490,22.0,8.0
3,13203841,21.0,4.0
4,15435022,21.0,1.0
...,...,...,...
40287,6509519,0.0,1.0
40288,17865804,0.0,1.0
40289,15848400,0.0,1.0
40290,4838598,0.0,1.0


In [18]:
# user별 question/answer count를 가지고 있는 데이터
with open('../data/df_qna.pkl', 'wb') as f:
    pickle.dump(df_qna, f)


In [19]:
# gephi 시각화용 데이터 
with open('../data/df_gephi.pkl', 'wb') as f:
    pickle.dump(df_gephi, f)

In [20]:
# 전체 데이터셋 
with open('../data/df.pkl', 'wb') as f:
    pickle.dump(df, f)